In [1]:
import findspark
findspark.init('/home/ubuntu/spark-3.3.1-bin-hadoop3')
findspark.find()

'/home/ubuntu/spark-3.3.1-bin-hadoop3'

In [2]:
from pyspark.sql import SparkSession

# The entry point into all functionality in Spark is the SparkSession class.
spark = (SparkSession
	.builder
	.appName("DS5110/CS5501: DDoS detection project")
	.master("spark://172.31.24.89:7077")
	.config("spark.executor.memory", "1024M")
	.getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/04/27 01:04:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Load csv data into dataframe

In [3]:
df = (spark.read
         .format("csv")
         .option("inferSchema", True)
         .option("header", True)
         .load("hdfs://172.31.24.89:9000/02-20-2018.csv"))

In [4]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator

### Data preprocessing

In [5]:
from pyspark.sql.functions import col, when
#drop unused columns, drop Nan values, convert string labels to integer
df = df.drop('Timestamp', 'Flow ID')
df = df.na.drop()
df = df.withColumn("Label", when(col("Label") == "Benign", 1).otherwise(0))

In [ ]:
df.show(2)

In [ ]:
#separate features and label columns

# Replace missing values in 'Label' with the most common value (mode)
# mode = labels.groupBy("Label").count().orderBy("count", ascending=False).first()["Label"]
# labels = labels.na.fill(value=mode, subset=["Label"])

In [ ]:
df.select("Src IP").show(10)

In [6]:
from pyspark.sql import functions as F
from pyspark.sql.types import LongType
import socket
import struct

# Define a UDF to convert IP to integer
def ip_to_int(ip):
    if ip:
        return struct.unpack("!I", socket.inet_aton(ip))[0]
    return None

# Register the UDF
ip_to_int_udf = F.udf(ip_to_int, LongType())

# Apply the UDF to transform the IP columns
df = df.withColumn('Src IP Int', ip_to_int_udf(F.col('Src IP')))
df = df.withColumn('Dst IP Int', ip_to_int_udf(F.col('Dst IP')))

In [7]:
df.show(2)

24/04/27 01:06:46 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------+--------+------------+--------+--------+-------------+------------+------------+---------------+---------------+---------------+---------------+----------------+---------------+---------------+---------------+----------------+---------------+-----------+-----------+-------------+------------+------------+------------+-----------+------------+-----------+-----------+-----------+-----------+------------+-----------+-----------+-----------+-------------+-------------+-------------+-------------+--------------+--------------+-----------+-----------+-----------+-----------+------------+-----------+-----------+------------+------------+------------+------------+------------+------------+--------------+------------+-------------+------------+----------------+----------------+--------------+--------------+----------------+--------------+--------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+-------

In [ ]:
label = df.select('Label').show(2)

In [8]:
df= df.drop('Src IP', 'Dst IP')
# label = df.select('Label')
# features = df.drop('Label')
label = df.columns[-3]
features= [col for col in df.columns if col != label]
# label.show()
# features.show(2)

In [13]:
df.show(2)

+--------+--------+--------+-------------+------------+------------+---------------+---------------+---------------+---------------+----------------+---------------+---------------+---------------+----------------+---------------+-----------+-----------+-------------+------------+------------+------------+-----------+------------+-----------+-----------+-----------+-----------+------------+-----------+-----------+-----------+-------------+-------------+-------------+-------------+--------------+--------------+-----------+-----------+-----------+-----------+------------+-----------+-----------+------------+------------+------------+------------+------------+------------+--------------+------------+-------------+------------+----------------+----------------+--------------+--------------+----------------+--------------+--------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+-----------------+----------------+

### Feature selection using PCA

In [9]:
from pyspark.ml.feature import PCA, VectorAssembler

In [32]:
assembler = VectorAssembler(inputCols=features, outputCol="features")
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")
pca = PCA(k=10, inputCol="scaledFeatures", outputCol="pcaFeatures")  # Reduce to 50 principal components

### Logistic regression 

In [ ]:

# model = pipeline.fit(df)
# data = model.transform(df)


In [33]:
# Select the features vector and the label column
# data = data.select("pcaFeatures", label)
# Split the data into training and test sets
trainData, testData = df.randomSplit([0.7, 0.3])

In [34]:
# Define and fit the Logistic Regression model
import time
start_time = time.time()
lr = LogisticRegression(featuresCol="pcaFeatures", labelCol=label, maxIter=10, regParam=0.3, elasticNetParam=0.8)
pipeline = Pipeline(stages=[assembler, scaler, pca, lr])
lrModel = pipeline.fit(trainData)
end_time = time.time()
print(f"Execution time: {end_time - start_time} seconds")

Py4JJavaError: An error occurred while calling o580.fit.
: breeze.linalg.NotConvergedException: 
	at breeze.linalg.svd$.breeze$linalg$svd$$doSVD_Double(svd.scala:120)
	at breeze.linalg.svd$Svd_DM_Impl$.apply(svd.scala:36)
	at breeze.linalg.svd$Svd_DM_Impl$.apply(svd.scala:35)
	at breeze.generic.UFunc.apply(UFunc.scala:46)
	at breeze.generic.UFunc.apply$(UFunc.scala:45)
	at breeze.linalg.svd$.apply(svd.scala:21)
	at org.apache.spark.mllib.linalg.distributed.RowMatrix.computePrincipalComponentsAndExplainedVariance(RowMatrix.scala:501)
	at org.apache.spark.mllib.feature.PCA.fit(PCA.scala:65)
	at org.apache.spark.ml.feature.PCA.fit(PCA.scala:93)
	at org.apache.spark.ml.feature.PCA.fit(PCA.scala:64)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4

In [ ]:
predictions = lrModel.transform(testData)

### Evaluation

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# Evaluate the model
evaluator = BinaryClassificationEvaluator(labelCol="Label")
accuracy = evaluator.evaluate(predictions)

precision_evaluator = MulticlassClassificationEvaluator(
    labelCol="Label",
    predictionCol="prediction",
    metricName="precisionByLabel")

# You can specify the label here if you have multiple classes and you are interested in one specific class
precision = precision_evaluator.evaluate(predictions)

# Evaluator for recall
recall_evaluator = MulticlassClassificationEvaluator(
    labelCol="Label",
    predictionCol="prediction",
    metricName="recallByLabel")

# As with precision, specify the label for recall calculation if needed
recall = recall_evaluator.evaluate(predictions)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
